In [1]:
from configs.default import _C as config # SYNTAX N.B. this imports configs > default.py 
from configs.default import update_config

from datasets import coco_parse
from datasets import flickr8k_parse

from keras import Model 
from keras.applications import VGG16
from tensorflow.python.client import device_lib

from models import image_preprocessing, transfer_models

import numpy as np
import os
import time

Using TensorFlow backend.


# Image encoding

Currently, an image encoder is built using VGG16 architecture pre-trained on imagenet database.

The features were obtained from the "fc2" layer - last fully-connected layer before the predictions layer.

Generated features in numpy arrays for both training and validation datasets were saved to .npy files.

### COCO dataset

In [2]:
config_file = "./configs/attn.yaml"
update_config(config, config_file)

In [3]:
if config.DATASET == 'Coco':
    if config.ATTENTION:
        features_file_train = "vgg16_coco_train_attn.npy"
        features_file_val = "vgg16_coco_val_attn.npy"
        features_file_test = "vgg16_coco_test_attn.npy"
    else:
        features_file_train = "vgg16_coco_train.npy"
        features_file_val = "vgg16_coco_val.npy"
        features_file_test = "vgg16_coco_test.npy"
    
    
    val_filenames_with_captions = coco_parse.get_image_filename_with_caption(config.PATH.ANNOTATIONS_PATH, 
                                                                             config.PATH.IMG_PATH, 
                                                                             train=False)

    val_filenames_with_all_captions = coco_parse.get_image_with_all_captions(val_filenames_with_captions)

    train_filenames_with_captions = coco_parse.get_image_filename_with_caption(config.PATH.ANNOTATIONS_PATH, 
                                                                               config.PATH.IMG_PATH,
                                                                               train=True)
    train_filenames_with_all_captions = coco_parse.get_image_with_all_captions(train_filenames_with_captions)

### Flickr8k dataset

In [4]:
if config.DATASET == 'Flickr8k':
    print("config.PATH.ANNOTATIONS_PATH",config.PATH.ANNOTATIONS_PATH)
    captions_file = os.path.join(config.PATH.ANNOTATIONS_PATH, "Flickr8k.token.txt")
    train_txt_path = os.path.join(config.PATH.ANNOTATIONS_PATH, "Flickr_8k.trainImages.txt")
    dev_txt_path = os.path.join(config.PATH.ANNOTATIONS_PATH, "Flickr_8k.devImages.txt")
    test_txt_path = os.path.join(config.PATH.ANNOTATIONS_PATH, "Flickr_8k.testImages.txt")
    
    if config.ATTENTION: # these are output files in numpy save binary format
        features_file_train = "vgg16_flickr8k_train_attn.npy"
        features_file_val = "vgg16_flickr8k_val_attn.npy"
        features_file_test = "vgg16_flickr8k_test_attn.npy"
    else:
        features_file_train = "vgg16_flickr8k_train.npy"
        features_file_val = "vgg16_flickr8k_val.npy"
        features_file_test = "vgg16_flickr8k_test.npy"

    filenames_with_all_captions = flickr8k_parse.generate_filenames_with_all_captions(captions_file, 
                                                                                      config.PATH.IMG_PATH)

# these are three dictionaries that are subsets of filenames_with_all_captions dictionary
# they are created based on the jpg names in Flickr_8k.trainImages.txt, etc.

    train_filenames_with_all_captions = flickr8k_parse.generate_set(train_txt_path, 
                                                                    filenames_with_all_captions,
                                                                    config.PATH.IMG_PATH)
    val_filenames_with_all_captions = flickr8k_parse.generate_set(dev_txt_path, 
                                                                  filenames_with_all_captions, 
                                                                  config.PATH.IMG_PATH)
    test_filenames_with_all_captions = flickr8k_parse.generate_set(test_txt_path, 
                                                                   filenames_with_all_captions, 
                                                                   config.PATH.IMG_PATH)

config.PATH.ANNOTATIONS_PATH C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/annotations/


In [5]:
print('Number of images in validation dataset: {}'.format(len(val_filenames_with_all_captions)))
print('Number of images in training dataset: {}'.format(len(train_filenames_with_all_captions)))
n=15
n1=-1
# [print(v) for i, v in enumerate(val_filenames_with_all_captions.items()) if i < n]
for (key, values) in val_filenames_with_all_captions.items():
    for value in values:
# prints key (filename) over and over with successive captions
        n1+=1
        if n1 < n:
            print (n1,key,value)

Number of images in validation dataset: 1000
Number of images in training dataset: 6000
0 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2090545563_a4e66ec76b.jpg the boy laying face down on a skateboard is being pushed along the ground by another boy .
1 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2090545563_a4e66ec76b.jpg Two girls play on a skateboard in a courtyard .
2 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2090545563_a4e66ec76b.jpg Two people play on a long skateboard .
3 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2090545563_a4e66ec76b.jpg Two small children in red shirts playing on a skateboard .
4 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2090545563_a4e66ec76b.jpg two young children on a skateboard going across a sidewalk


In [6]:
### encode features for validation images
start = time.time()
printswitch=False
val_transfer_values = transfer_models.use_pretrained_model_for_images(val_filenames_with_all_captions,
                                                                      config.ATTENTION,printswitch, 
                                                                      batch_size=config.ENCODER.BATCH_SIZE)
time_val = time.time() - start

use_pretrained_model_for_image - filenames_with_all_captions C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2090545563_a4e66ec76b.jpg










_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)    

filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2648310638_d6c576b5e4.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2726262796_03bd63a155.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3631023049_8a85bab19f.jpg
35% of images processed
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1801663973_5ad393caeb.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2679565682_91ecd283ff.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3163068926_d28ed3ef53.jpg
40% of images processed
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2598012140_832863fcb9.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3154152744_4e93ec8a62.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3331525712_af1dcc47f2.jpg
45% of images processed
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1370773415_967b1ffde1.jpg
filename

In [7]:
print('Validation dataset encoding took {:.1f} minutes'.format(time_val / 60))

Validation dataset encoding took 4.0 minutes


In [8]:
### save features for validation images
print("val_transfer_values",val_transfer_values)
transfer_models.save_features(val_transfer_values, config.PATH.FEATURES_PATH, features_file_val)

val_transfer_values [[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    6.66217864e-01 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    3.17280471e-01 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    4.21574146e-01 0.00000000e+00]
   ...
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    1.48402369e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    1.09586990e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    7.49951601e-01 0.00000000e+00]]

  [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    4.33700800e-01 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    2.08732486e-01 0.00000000e+00]
   ...
   [0.00000000e+00 0.00000000e+00 0.0

Array was saved to ./cnn_features/vgg16_flickr8k_val_attn.npy


In [9]:
### encode features for training images
print("val_transfer_values.shape",val_transfer_values.shape)
start = time.time()
printswitch=False
train_transfer_values = transfer_models.use_pretrained_model_for_images(train_filenames_with_all_captions, 
                                                                        config.ATTENTION,printswitch, 
                                                                        batch_size=config.ENCODER.BATCH_SIZE)
time_train = time.time() - start

val_transfer_values.shape (1000, 14, 14, 512)
use_pretrained_model_for_image - filenames_with_all_captions C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2513260012_03d33305cf.jpg
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv

filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2923825744_ca125353f0.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3737492755_bcfb800ed1.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/358559906_d5f3f584f4.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3465473743_7da0c5d973.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3454988449_1de1ef4f20.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3110174991_a4b05f8a46.jpg
10% of images processed
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2873431806_86a56cdae8.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/241345656_861aacefde.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/860928274_744d14f198.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/307994435_592f933a6d.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/I

filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/617038406_4092ee91dd.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/172097783_292c5413d8.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2230067846_74046b89d3.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/272546805_536c719648.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2385871317_44cde2f354.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3103587323_7f093d5b90.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/437917001_ae1106f34e.jpg
35% of images processed
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3581451227_618854cea4.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2800004913_c8394ba332.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/407678652_1f475acd65.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Im

filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3544312930_3a0b8d70c1.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3028404926_2bd27e3e83.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/17273391_55cfc7d3d4.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3096918227_f9d26a7db2.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1262077938_8b9516c273.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2422018883_336519b5c6.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1079274291_9aaf896cc1.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3654869593_c8599a8e20.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3514184232_b336414040.jpg
60% of images processed
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3049770416_0fb1954315.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k

filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2346189044_546ed84aa9.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3268407162_6274e0f74f.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1801063894_60bce29e19.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2752043092_f48ebfeaa2.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3353278454_2f3a4d0bbc.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2492258999_5764124bba.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3474985382_26e1560338.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3048461682_e89f81b1c7.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/774009278_8e75b7d498.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/397451339_76a84bd310.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3587009091_37188

In [10]:
print('Training dataset encoding took {:.1f} minutes'.format(time_train / 60))

Training dataset encoding took 26.0 minutes


In [11]:
transfer_models.save_features(train_transfer_values, config.PATH.FEATURES_PATH, features_file_train)

Array was saved to ./cnn_features/vgg16_flickr8k_train_attn.npy


In [12]:
### encode features for training images
start = time.time()
printswitch=True
test_transfer_values = transfer_models.use_pretrained_model_for_images(test_filenames_with_all_captions, 
                                                                       config.ATTENTION,printswitch, 
                                                                       batch_size=config.ENCODER.BATCH_SIZE)
print("test_transfer_values.shape",test_transfer_values.shape)
time_train = time.time() - start

use_pretrained_model_for_image - filenames_with_all_captions C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3385593926_d3e9c21170.jpg
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584   

filename, i00 42 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2921094201_2ed70a7963.jpg
filename, i00 43 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2553550034_5901aa9d6c.jpg
filename, i00 44 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3045613316_4e88862836.jpg
filename, i00 45 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2706766641_a9df81969d.jpg
filename, i00 46 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/510531976_90bbee22a2.jpg
filename, i00 47 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/485245061_5a5de43e20.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3070011270_390e597783.jpg
first_i 48 last_i 64
filename, i00 48 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3070011270_390e597783.jpg
filename, i00 49 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1352410176_af6b139734.jpg
filename, i00 50 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1131932671_c8d1

filename, i00 121 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/401079494_562454c4d6.jpg
filename, i00 122 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2396691909_6b8c2f7c44.jpg
filename, i00 123 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3243588540_b418ac7eda.jpg
filename, i00 124 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3592992234_6d3fe58a70.jpg
filename, i00 125 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1417031097_ab656bc4bd.jpg
filename, i00 126 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1122944218_8eb3607403.jpg
filename, i00 127 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3149919755_f9272b10b3.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2682382530_f9f8fd1e89.jpg
10% of images processed
first_i 128 last_i 144
filename, i00 128 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2682382530_f9f8fd1e89.jpg
filename, i00 129 C:/Users/MStopa/ImageCaptioning/data

filename, i00 200 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/751109943_2a7f8e117f.jpg
filename, i00 201 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3121521593_18f0ec14f7.jpg
filename, i00 202 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1432179046_8e3d75cf81.jpg
filename, i00 203 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3234115903_f4dfc8fc75.jpg
filename, i00 204 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3497224764_6e17544e0d.jpg
filename, i00 205 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2878272032_fda05ffac7.jpg
filename, i00 206 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1536774449_e16b1b6382.jpg
filename, i00 207 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2228022180_9597b2a458.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2708686056_1b8f356264.jpg
first_i 208 last_i 224
filename, i00 208 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/270

filename, i00 284 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3080056515_3013830309.jpg
filename, i00 285 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1425069308_488e5fcf9d.jpg
filename, i00 286 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/261490838_2f3ac98b12.jpg
filename, i00 287 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2926233397_71e617f3a3.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2963573792_dd51b5fbfb.jpg
first_i 288 last_i 304
filename, i00 288 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2963573792_dd51b5fbfb.jpg
filename, i00 289 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3416091866_a96003d652.jpg
filename, i00 290 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2431470169_0eeba7d602.jpg
filename, i00 291 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3099923914_fd450f6d51.jpg
filename, i00 292 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/524

...

filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2922222717_12195af92d.jpg
60% of images processed
first_i 608 last_i 624
filename, i00 608 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2922222717_12195af92d.jpg
filename, i00 609 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3191135894_2b4bdabb6d.jpg
filename, i00 610 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/700884207_d3ec546494.jpg
filename, i00 611 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2196846255_2c1635359a.jpg
filename, i00 612 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3474406285_01f3d24b71.jpg
filename, i00 613 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/448252603_7d928c900e.jpg
filename, i00 614 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2860872588_f2c7b30e1a.jpg
filename, i00 615 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/880220939_0ef1c37f1f.jpg
filename, i00 616 C:/Users/MStopa/ImageCaptioning/datase

filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1311388430_4ab0cd1a1f.jpg
first_i 688 last_i 704
filename, i00 688 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1311388430_4ab0cd1a1f.jpg
filename, i00 689 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/522063319_33827f1627.jpg
filename, i00 690 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/384577800_fc325af410.jpg
filename, i00 691 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3159995270_17334ccb5b.jpg
filename, i00 692 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2229179070_dc8ea8582e.jpg
filename, i00 693 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1764955991_5e53a28c87.jpg
filename, i00 694 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/670609997_5c7fdb3f0b.jpg
filename, i00 695 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/86412576_c53392ef80.jpg
filename, i00 696 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2608288

filename, i00 767 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1220401002_3f44b1f3f7.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3564543247_05cdbc31cf.jpg
first_i 768 last_i 784
filename, i00 768 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3564543247_05cdbc31cf.jpg
filename, i00 769 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3385246141_a263d1053e.jpg
filename, i00 770 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3334537556_a2cf4e9b9a.jpg
filename, i00 771 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2909955251_4b326a46a7.jpg
filename, i00 772 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2247889670_413db8094b.jpg
filename, i00 773 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3186073578_6e115f45f5.jpg
filename, i00 774 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3192069971_83c5a90b4c.jpg
filename, i00 775 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/34

filename, i00 843 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/113678030_87a6a6e42e.jpg
filename, i00 844 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2120411340_104eb610b1.jpg
filename, i00 845 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2450453051_f1d4a78ab4.jpg
filename, i00 846 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2831217847_555b2f95ca.jpg
filename, i00 847 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3192266178_f9bf5d3dba.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/326456451_effadbbe49.jpg
85% of images processed
first_i 848 last_i 864
filename, i00 848 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/326456451_effadbbe49.jpg
filename, i00 849 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2663794355_e726ec7e05.jpg
filename, i00 850 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3364151356_eecd07a23e.jpg
filename, i00 851 C:/Users/MStopa/ImageCaptioning/datase

filename, i00 927 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/493621130_152bdd4e91.jpg
filename C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3332467180_d72f9b067d.jpg
first_i 928 last_i 944
filename, i00 928 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/3332467180_d72f9b067d.jpg
filename, i00 929 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2981702521_2459f2c1c4.jpg
filename, i00 930 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/461505235_590102a5bf.jpg
filename, i00 931 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/1490213660_9ea45550cf.jpg
filename, i00 932 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/300577375_26cc2773a1.jpg
filename, i00 933 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2549968784_39bfbe44f9.jpg
filename, i00 934 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/2075321027_c8fcbaf581.jpg
filename, i00 935 C:/Users/MStopa/ImageCaptioning/datasets/Flickr8k/Images/15478

In [13]:
transfer_models.save_features(test_transfer_values, config.PATH.FEATURES_PATH, features_file_test)

Array was saved to ./cnn_features/vgg16_flickr8k_test_attn.npy
